## Week 11 Workshop

In this week we are going to work with the multi-modal model ESM-3, which can do a variety of protein design tasks. We will also use AlphaFold 3 to predict structures.

To work through all the elements of this workshop, you will need free accounts on both the ESM Forge and the AlphaFold Server:

- ESM Forge: https://forge.evolutionaryscale.ai/
- AlphaFold Server: https://alphafoldserver.com/

As always, we start by importing the dependencies.

In [1]:
import esm
from esm.sdk.api import ESMProtein, GenerationConfig
import py3Dmol
import biotite.structure.io.pdbx as pdbx
from biotite.structure.io.pdb import PDBFile
from biotite.structure import superimpose, rmsd
from io import StringIO
import warnings
from getpass import getpass

The ESM Forge uses an API Key system through which you can interact from your Python scripts with their server. To generate a key, log in to your account, click on "API Keys" and create a new key. Save the key in a secure location. It is like a password.

In [2]:
# Enter your API key here. Remeber you need to keep it confidential.
# Never add your key to a github repo or post it publicly in any way.
api_key = getpass("Your EvolutionaryScale API key: ")

Now we can create the model object. Unlike in previous workshops, we are not downloading the model, we are simply connecting to the ESM Forge server.

In [3]:
model = esm.sdk.client("esm3-medium-2024-08", token=api_key)

## ESM-3 for loop rebuilding

We will use ESM-3 for loop rebuilding, where we take an existing protein, mask some portions of it, and then ask ESM-3 to rewrite the masked portions. As a first step, we need a helper function that makes it easy to mask portions of a protein sequence.

In [4]:
def mask_sequence(protein, retain=None, replace=None):
    """
    Mask parts of a protein sequence with underscores.
    
    Parameters:
    -----------
    protein : str
        The amino acid sequence
    retain : str, optional
        Positions to retain (1-indexed), e.g., "1,2,6-10,22-45"
    replace : str, optional
        Positions to replace (1-indexed), e.g., "1-4,15,20-25"
    
    Returns:
    --------
    str
        The masked sequence
    
    Note: If both retain and replace are specified, retain takes precedence.
    """
    def parse_positions(pos_string):
        positions = set()
        for part in pos_string.split(','):
            if '-' in part:
                start, end = map(int, part.split('-'))
                positions.update(range(start, end + 1))
            else:
                positions.add(int(part))
        return positions
    
    masked = list(protein)
    
    if retain:
        keep = parse_positions(retain)
        masked = [ch if i+1 in keep else '_' for i, ch in enumerate(masked)]
    elif replace:
        rep = parse_positions(replace)
        masked = ['_' if i+1 in rep else ch for i, ch in enumerate(masked)]
    
    return ''.join(masked)


# Example usage:
protein = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQC"

# Example 1: Retain specific positions
result1 = mask_sequence(protein, retain="1,2,6-10,22-45")
print("Retain 1,2,6-10,22-45:")
print(result1)

# Example 2: Replace specific positions (e.g., replace first 4)
result2 = mask_sequence(protein, replace="1-4")
print("\nReplace 1-4:")
print(result2)

# Example 3: Retain positions 5-15
result3 = mask_sequence(protein, retain="5-15")
print("\nRetain 5-15:")
print(result3)

# Example 4: Replace multiple ranges
result4 = mask_sequence(protein, replace="1-10,20-30,50-60")
print("\nReplace 1-10,20-30,50-60:")
print(result4)

Retain 1,2,6-10,22-45:
MS___ELFTG___________VNGHKFSVSGEGEGDATYGKLTLK_________________________

Replace 1-4:
____EELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQC

Retain 5-15:
____EELFTGVVPIL_______________________________________________________

Replace 1-10,20-30,50-60:
__________VVPILVELD___________GEGEGDATYGKLTLKFICT___________VTTFSYGVQC


We will be working with GFP, whose structure is available under the PDB ID 1B9C. You can find it on the RCSB website here: https://www.rcsb.org/3d-view/1B9C/1

I have taken the sequence from GFP and refolded with AlphaFold 3 for easier comparison with the sequences we will generate with ESM-3. This structure is available in the repository under the name `GFP_AF3.cif`. Let's begin by visualizing this structure, and specifically highlighting the parts that we will rebuild. We will rebuild residues 129-147 and 188-198.

In [5]:
# Visualize the masked regions
view = py3Dmol.view(width=600, height=400)
view.addModel(open('GFP_AF3.cif').read(), 'cif')

view.setStyle({}, {})  # hide everything
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'lightblue'}})
view.setStyle({'chain': 'A', 'resi': ['129-147', '188-198']}, {'cartoon': {'color': 'tomato'}})

view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now we are ready to generate our masked input sequence for ESM-3.

In [6]:
protein = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
masked = mask_sequence(protein, replace="129-147,188-198")
print(masked)

MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGI___________________HNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTP___________HYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


Next we input this masked sequence into ESM-3 and fill the masked parts.

In [7]:
protein = ESMProtein(sequence=masked)

# Generate the protein sequence
protein = model.generate(
    protein,
    GenerationConfig(
        track="sequence",  # specify that we want to generate a sequence
        num_steps=8,       # number of steps for iterative decoding
        temperature=0.7    # controls diversity vs. perplexity
    )
)

# The `protein` object now contains the generated sequence.
# You can access it via `protein.sequence`
print(protein.sequence)

MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIGFSPDGKILSGKITGIEPSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPLSGKPIELPEPHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


This is our generated sequence. To figure out how it may look as a folded protein, we can generate a structure with AlphaFold 3 and then visualize.

Every time you run this code you will get a different sequence. Here is the sequence that corresponds to the predicted structure in this repository: 

In [8]:
print(
    "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIGFSPDGKILSGKITGIEPSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPLSGKPIELPEPHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
)

MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIGFSPDGKILSGKITGIEPSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPLSGKPIELPEPHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK


## Comparison of predicted structures

Go to the [AlphaFold Server](https://alphafoldserver.com/) and create a job to fold your predicted protein. Once the job is done, download the output into the AF3 folder in this repository and unzip. Then we can align the predicted structure with the GFP structure from AlphaFold for the original GFP sequence and see how similar or different they are.

But before we can do the visualization, we need a function that can align and visualize two structures.

In [9]:
# suppress irrelevant warnings
warnings.filterwarnings('ignore', message='Attribute .* not found within')

def align_and_visualize_cif(cif_file1, cif_file2, chain1='A', chain2='A', highlight=None):
    """
    Align two CIF files and visualize the overlay.
    
    Parameters:
    -----------
    cif_file1 : str
        Reference CIF file
    cif_file2 : str
        CIF file to align
    chain1, chain2 : str
        Chain IDs to use for alignment
    """
    # Read CIF files
    structure1 = pdbx.get_structure(pdbx.CIFFile.read(cif_file1), model=1)
    structure2 = pdbx.get_structure(pdbx.CIFFile.read(cif_file2), model=1)
    
    # Get CA atoms for alignment
    ca1 = structure1[(structure1.chain_id == chain1) & (structure1.atom_name == 'CA')]
    ca2 = structure2[(structure2.chain_id == chain2) & (structure2.atom_name == 'CA')]
    
    # Match lengths
    n_atoms = min(len(ca1), len(ca2))
    ca1 = ca1[:n_atoms]
    ca2 = ca2[:n_atoms]
    
    # Superimpose CA atoms to get transformation
    ca2_aligned, transformation = superimpose(ca1, ca2)
    
    # Apply transformation to full structure
    structure2_aligned = structure2.copy()
    structure2_aligned.coord = transformation.apply(structure2.coord)
    
    # Print alignment info
    print(f"RMSD: {rmsd(ca1, ca2_aligned):.2f} Å ({n_atoms} CA atoms)")
    
    # Convert to PDB format for visualization
    pdb1 = StringIO()
    pdb_file1 = PDBFile()
    pdb_file1.set_structure(structure1)
    pdb_file1.write(pdb1)
    
    pdb2 = StringIO()
    pdb_file2 = PDBFile()
    pdb_file2.set_structure(structure2_aligned)
    pdb_file2.write(pdb2)
    
    # Visualize
    view = py3Dmol.view(width=600, height=400)
    view.addModel(pdb1.getvalue(), 'pdb')
    view.addModel(pdb2.getvalue(), 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'lightblue', 'opacity': .8}})
    view.setStyle({'model': 1}, {'cartoon': {'color': 'khaki'}})
    if highlight is not None:
        view.setStyle({'model': 1, 'resi': highlight}, {'cartoon': {'color': 'tomato'}})
    view.zoomTo()
    
    return view

Now we can generate the visualization.

In [16]:
view = align_and_visualize_cif(
    'GFP_AF3.cif',                                     # original structure
    'AF3/fold_gfp_esm3_129_147_188_198_model_0.cif',   # rebuilt structure
    highlight = ['129-147', '188-198']         # optional: highlight rebuilt loops
)
view.show()

RMSD: 1.00 Å (238 CA atoms)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

We can also align and visualize multiple AlphaFold 3 models to see how they compare.

In [18]:
view = align_and_visualize_cif(
    'AF3/fold_gfp_esm3_129_147_188_198_model_0.cif',
    'AF3/fold_gfp_esm3_129_147_188_198_model_4.cif'
)
view.show()

RMSD: 1.02 Å (238 CA atoms)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Exercises

- Mask different parts of the GFP sequence and see what results you get.

- Change the temperature or the number of steps in the `model.generate()` call and see what effects they have. 

- See how much you can delete from the GFP sequence before results start falling apart. For example, what happens when you mask from position 188 all the way till the end (position 238)?